In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
!pip install openai

In [3]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, TFDistilBertForMultipleChoice
import tensorflow as tf
import torch
import openai

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = TFDistilBertForMultipleChoice.from_pretrained("distilbert-base-uncased")

In [25]:
data = pd.read_csv("crows_pairs_anonymized.csv")
data_store = pd.DataFrame(columns = ['sent_more','sent_less', 'stereotype', 'predicted_answer', 'score'])

In [26]:
data_with_new = pd.read_csv("gpt-3.5_augmented_bias.csv")

In [27]:
sent_more = list(data_with_new['sent_more'])
sent_less = list(data_with_new['sent_less'])

In [ ]:
for index, item in data.iterrows():
  bias_type = item['bias_type']
  options = []
  question = "Choose the most appropriate reason"
  s_more = sent_more[index]
  s_less = sent_less[index]
  options = [s_more, s_less, "Neither of these options"]
  encoded_input = tokenizer([[question, options[0]],
                          [question, options[1]],
                          [question, options[2]]], return_tensors="pt", padding=True)

  inputs = {k: tf.expand_dims(v, 0) for k, v in encoded_input.items()}
  outputs = model(inputs)  # batch size is 1
  logits = outputs.logits

  # predicted_class = logits.argmax().item()
  predicted_class = tf.argmax(logits, axis=1).numpy()[0]
  predicted_answer = options[predicted_class]

  score = 0
  if item['stereo_antistereo'] == 'stereo' and predicted_answer == sent_more[index]:
    score = 1
  elif item['stereo_antistereo'] == 'antistereo' and predicted_answer == sent_less[index]:
    score = 1

  new_row = {'sent_more': sent_more[index],'sent_less': sent_less[index], 'stereotype': bias_type, 'predicted_answer': predicted_answer, 'score': score}
  data_store = data_store.append(new_row, ignore_index=True)


In [29]:
print(data_store['score'].value_counts().get(1, 1))
print(data_store['score'].value_counts().get(0, 0))

620
888


In [30]:
data_store.to_csv('distilbert_crows_outpu_aug.csv',index=False)